# ¿Que más se podría hacer?

- Test con data augmentation y la LSTM
- Usar los tokenizadores usados en los transformers para ver como funcionan
- TSNE de embeddings y analizar si hay interpretación
- Probar con otras encimas o proteinas
- Usar los embeddings entrenados para analizar resultados de proteinas o encimas con menos data
- Entrenar una red neuronal con los features (fingerprints por ejemplo) y comparar los resultados con los embeddings


# Tome cualquiera de estas propuestas o alguna suya y desarrolle

# Elección: TSNE de embeddings y analizar si hay interpretación

## Preparacion e inicializacion

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, Embedding, Bidirectional, Activation, Input, Conv1D, MaxPool1D, Concatenate, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
from datagen import smiles_dict, DataGenerator

In [2]:
df = pd.read_csv('data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

In [3]:
max_len_idx = df['canonical_smiles'].apply(len).argmax()
min_len_idx = df['canonical_smiles'].apply(len).argmin()
max_sequence_len = len(df['canonical_smiles'].iloc[max_len_idx]) + 20
X = df['canonical_smiles'].values
y = df['pIC50'].values
vocab_size = len(smiles_dict)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Genera

In [16]:
dgen_train = DataGenerator(X_train, y_train, seq_length=max_sequence_len, batch_size=128, data_augmentation=True)

dgen_test = DataGenerator(X_test, y_test, seq_length=max_sequence_len, batch_size=128, data_augmentation=False)

In [6]:
## Red

In [7]:
def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [8]:
# Hyperparametros
learning_rate=0.01 #0.01   0.05    1e-3
epochs = 130     #50    30
batch_size=128     #64      32

lstm = 100 #64   20
dense_size = 512 #100 #50
drops = 0.8   #0.4
embeddings = 128

In [9]:
import mlflow.keras
mlflow.autolog()
#! mlflow ui

2023/02/05 10:19:31 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2023/02/05 10:19:32 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [10]:
model = Sequential()
model.add(Embedding(vocab_size + 1, embeddings, input_length=max_sequence_len))
model.add(Bidirectional(LSTM(lstm, return_sequences=False, return_state=False, activation='tanh')))
model.add(Dense(dense_size))
model.add(BatchNormalization())
model.add(Dropout(drops))
model.add(Dense(dense_size))
model.add(BatchNormalization())
model.add(Dropout(drops))
model.add(Activation('relu'))
model.add(Dense(1, activation='linear'))
model.summary()

2023-02-05 10:19:32.175492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 227, 128)          5504      
                                                                 
 bidirectional (Bidirectiona  (None, 200)              183200    
 l)                                                              
                                                                 
 dense (Dense)               (None, 512)               102912    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               2

In [11]:
model.compile(optimizer=RMSprop(learning_rate=learning_rate), loss='mse', metrics=[R2])

In [12]:
# callbacks
mcp = ModelCheckpoint('models/best_model_{epoch}', save_best_only=True, save_format="h5")
earlyStopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, restore_best_weights=True)
reduceLR = ReduceLROnPlateau(monitor ='val_loss', factor=0.2, patience=15, verbose=1, mode='auto', min_delta=0.01, cooldown=0, min_lr=1e-8)

In [13]:
%%time
history = model.fit(dgen_train, epochs=epochs, batch_size=batch_size, validation_data=augm_test, callbacks=[mcp, reduceLR])

2023/02/05 10:19:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd3d62469faba470f85c890567e7aa8cb', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/130
37/37 [==============================] - ETA: 0s - loss: 10.0353 - R2: -2.9159

INFO:tensorflow:Assets written to: models/best_model_1/assets


INFO:tensorflow:Assets written to: models/best_model_1/assets


37/37 [==============================] - 38s 929ms/step - loss: 10.0353 - R2: -2.9159 - val_loss: 8.4135 - val_R2: -2.2905 - lr: 0.0100
Epoch 2/130
37/37 [==============================] - 26s 706ms/step - loss: 5.9044 - R2: -1.2806 - val_loss: 14.6804 - val_R2: -5.0883 - lr: 0.0100
Epoch 3/130
37/37 [==============================] - ETA: 0s - loss: 5.3317 - R2: -1.0909

...

37/37 [==============================] - 35s 944ms/step - loss: 1.1151 - R2: 0.5737 - val_loss: 1.1504 - val_R2: 0.5902 - lr: 0.0020


2023/02/05 11:26:53 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: unsupported operand type(s) for *: 'slice' and 'int'
2023/02/05 11:26:53 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/cc/hfx_w22x28j8f0l38h8ww1zc0000gn/T/tmp9x6p1hsg/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/cc/hfx_w22x28j8f0l38h8ww1zc0000gn/T/tmp9x6p1hsg/model/data/model/assets
2023/02/05 11:27:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/cc/hfx_w22x28j8f0l38h8ww1zc0000gn/T/tmp9x6p1hsg/model, flavor: tensorflow), fall back to return ['tensorflow==2.11.0']. Set logging level to DEBUG to see the full traceback.
2023/02/05 11:27:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


CPU times: user 3h 27min 40s, sys: 2h 23min 28s, total: 5h 51min 8s
Wall time: 1h 7min 34s


In [19]:
X_test_eval = []
y_t_eval = []
for X_t, y_t in dgen_test:
    X_test_eval = X_test_eval + [list(t) for t in X_t]
    y_t_eval = y_t_eval + list(y_t)
X_test_eval = np.array(X_test_eval)
y_test = np.array(y_t_eval)

y_pred = model.predict(X_test_eval)

37/37 [==============================] - 2s 45ms/step


In [21]:
r2 = 1 - ((y_test - y_pred.reshape(-1)) ** 2).sum() / ((y_test - y_test.mean()) ** 2).sum()
r2

0.5898812204959437

In [22]:
model.save('005/lstm_augment.h5', )

# Conclusion
No mejoró, le llevó mas iteraciones llegar a valores de R2 similares, la loss fue más alta.

In [23]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

date and time = 05/02/2023 11:54:06
